In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "loan_status", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('Resources\LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df = pd.get_dummies(df, columns=["loan_status", "home_ownership", "verification_status", "application_type"])


df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App
0,35000.0,0.1308,797.80,125890.0,30.48,0.0,0.0,14.0,0.0,32363.0,...,0,0,1,0,0,0,0,1,1,0
1,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,0,0,0,1,0,1,0,1,0
2,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,0,1,0,0,0,0,1,1,0
3,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,0,1,0,0,0,0,1,1,0
4,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,0,0,0,1,0,0,1,1,0


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.copy()
X = X.drop("loan_status_high_risk", axis=1)
X = X.drop("loan_status_low_risk", axis=1)

# Create our target
y = df["loan_status_high_risk"]
y

0        0
1        0
2        0
3        0
4        0
        ..
70418    0
70419    0
70420    0
70421    0
70422    0
Name: loan_status_high_risk, Length: 70423, dtype: uint8

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_Fully Paid,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App
count,70423.000000,70423.000000,70423.000000,7.042300e+04,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,...,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000
mean,16621.372989,0.128063,479.478688,8.843486e+04,21.745274,0.217244,0.500632,12.583787,0.125939,17549.650966,...,0.022493,0.009216,0.526518,0.106939,0.357326,0.477770,0.374111,0.148119,0.860898,0.139102
std,10282.268052,0.048339,288.457221,1.218578e+05,20.429814,0.718822,0.760803,6.028804,0.336711,21880.270487,...,0.148280,0.095556,0.499300,0.309039,0.479216,0.499509,0.483896,0.355221,0.346056,0.346056
min,1000.000000,0.060000,30.640000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,264.930000,5.000000e+04,13.840000,0.000000,0.000000,8.000000,0.000000,6240.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,402.890000,7.300000e+04,19.710000,0.000000,0.000000,11.000000,0.000000,12014.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,24000.000000,0.155700,646.740000,1.040000e+05,26.620000,0.000000,1.000000,16.000000,0.000000,21657.500000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,9.000000e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [9]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=78)
rf_model = rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

In [10]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)

In [11]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,14656,2855
Actual 1,41,54


In [12]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,14656,2855
Actual 1,41,54


Accuracy Score : 0.7026903390049563
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.84      0.57      0.91      0.69      0.49     17511
          1       0.02      0.57      0.84      0.04      0.69      0.46        95

avg / total       0.99      0.84      0.57      0.91      0.69      0.49     17606



In [13]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.06193388622023243, 'total_pymnt'),
 (0.06160958933100059, 'last_pymnt_amnt'),
 (0.059728042785636565, 'total_rec_prncp'),
 (0.05907450001337356, 'total_rec_int'),
 (0.05008047464160505, 'total_pymnt_inv'),
 (0.03559291316539752, 'int_rate'),
 (0.01815988485264555, 'revol_bal'),
 (0.01787701378455848, 'dti'),
 (0.017874486327278876, 'out_prncp'),
 (0.01768035302489635, 'mths_since_recent_inq'),
 (0.017577458362652003, 'out_prncp_inv'),
 (0.017221290524472656, 'installment'),
 (0.016775407810089054, 'mo_sin_old_il_acct'),
 (0.01671425692166349, 'annual_inc'),
 (0.01660786028357718, 'mo_sin_old_rev_tl_op'),
 (0.01618800248175096, 'total_rev_hi_lim'),
 (0.015856277853010674, 'il_util'),
 (0.015725741747927624, 'total_bc_limit'),
 (0.015713312138696072, 'tot_hi_cred_lim'),
 (0.015411314618784856, 'all_util'),
 (0.015220086672732872, 'max_bal_bc'),
 (0.014982296895774377, 'bc_open_to_buy'),
 (0.014784711527834359, 'tot_cur_bal'),
 (0.014440618738641558, 'bc_util'),
 (0.01413769939473701,

### Easy Ensemble AdaBoost Classifier

In [14]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ab_model = BalancedRandomForestClassifier(n_estimators=100, random_state=78)
ab_model = ab_model.fit(X_train, y_train)
predictions = ab_model.predict(X_test)

In [15]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)

In [16]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,14656,2855
Actual 1,41,54


In [17]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,14656,2855
Actual 1,41,54


Accuracy Score : 0.7026903390049563
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.84      0.57      0.91      0.69      0.49     17511
          1       0.02      0.57      0.84      0.04      0.69      0.46        95

avg / total       0.99      0.84      0.57      0.91      0.69      0.49     17606

